In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import re
from nltk.corpus import stopwords
ENG_STOP = stopwords.words("ENGLISH")
from sklearn.decomposition import TruncatedSVD
from copkmeans.cop_kmeans import cop_kmeans
from nltk.stem import *
import gensim
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
import spacy

In [72]:
df_la = pd.read_pickle("LATIMES_df")
df_la["From"] = "LA Times"
df_fbis = pd.read_pickle("df_1000_docs")
df_fbis["From"] = "FBIS"
df_ft = pd.read_pickle("FT_df")
df_ft["From"] = "FT"
df_la = df_la.sample(n=500,axis=0,random_state=11)
df_fbis = df_fbis.sample(n=500,axis=0,random_state=11)
df_ft = df_ft.sample(n=500,axis=0,random_state=11)
df = pd.concat([df_la,df_fbis,df_ft],axis=0)

In [73]:
stemmer = porter.PorterStemmer()

In [122]:
df = df[df["Text"]!="None"]

In [124]:
df.to_pickle("Raw_Df_130622")

In [125]:
len(df.Text.to_list())

1478

In [126]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([str(token.lemma_) for token in doc if token.pos_ in allowed_postags])
    return texts_out
def cleanhtml(raw_html):
    CLEANR = re.compile('<.*?>') 
    cleantext = re.sub(CLEANR, '', raw_html)
    cleantext = cleantext.replace("/"," ").replace("(","").replace(")","")
    res = []
    l_res = []
    for word in cleantext.split(" "):
        word = word.lower()
        if word.isalpha() and word not in ENG_STOP:
            res.append(word)
    return res
data = df.Text.to_list()
cleaned_data = [cleanhtml(doc) for doc in data]
cleaned_data = lemmatization(cleaned_data, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [127]:
pickle.dump(cleaned_data,open("cleaned_data_130622_F.pkl","wb"))

In [128]:
def get_bow(text,max_features):
    vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False,max_features=max_features)
    bow = vectorizer.fit_transform(text)
    text_d = bow.toarray()
    temp = pd.DataFrame(text_d, columns = vectorizer.get_feature_names())
    return temp

def get_tfidf(text,max_features):
    vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False,max_features=max_features)
    bow = vectorizer.fit_transform(text)
    text_d = bow.toarray()
    temp = pd.DataFrame(text_d, columns = vectorizer.get_feature_names())
    return temp

In [9]:
GoogleModel = gensim.models.KeyedVectors.load_word2vec_format(r"\GoogleNews-vectors-negative300.bin.gz", binary=True,)


In [129]:
# Defining a function which takes text input and returns one vector for each sentence
def FunctionText2Vec(GoogleModel = gensim.models.KeyedVectors.load_word2vec_format(r"\GoogleNews-vectors-negative300.bin.gz", binary=True,)inpTextData, cleaned_data,max_features):
    CountVectData=get_bow(cleaned_data,max_features=max_features)
    
    # Creating empty dataframe to hold sentences
    W2Vec_Data=pd.DataFrame()
    WordsVocab=CountVectData.columns[:]
    # Looping through each row for the data
    for i in range(CountVectData.shape[0]):

        # initiating a sentence with all zeros
        Sentence = np.zeros(300)

        # Looping thru each word in the sentence and if its present in 
        # the Word2Vec model then storing its vector
        for word in WordsVocab[CountVectData.iloc[i,:]>=1]:
            #print(word)
            if word in GoogleModel.key_to_index.keys():    
                Sentence=Sentence+GoogleModel[word]
        # Appending the sentence to the dataframe
        W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]), ignore_index=True)
    return W2Vec_Data

In [143]:
# Calling the function to convert all the text data to Word2Vec Vectors
Data=FunctionText2Vec(df['Text'], cleaned_data,max_features=10000)
# Checking the new representation for sentences

In [144]:
Data.columns = ["W2Vec_"+str(i) for i in range(300)]


In [145]:
data_tfidf = get_tfidf(cleaned_data,max_features=10000)


In [146]:
Data = pd.concat([Data,data_tfidf],axis=1)

In [18]:
kw_model = KeyBERT('all-MiniLM-L6-v2')


In [134]:
# Finding the key phrase for each document
keywords = kw_model.extract_keywords(df["Text"].to_list(), keyphrase_ngram_range=(3, 3), stop_words='english',
                              use_maxsum=True, nr_candidates=30, top_n=1)

C:\Users\rohee\anaconda3\lib\site-packages\keybert\_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn(
1478it [01:33, 15.78it/s]


In [137]:
# Embedding the key phrase using Sentence Transformer
model = SentenceTransformer('all-MiniLM-L6-v2')
kw = keywords
emb = []
for k in kw:
    emb.append(model.encode(k[0][0]))

In [138]:
kp_df = pd.DataFrame(emb, columns=["KP_"+str(i+1) for i in range(len(emb[0]))])
#kp_df_scaled = min_max_scaler.fit_transform(kp_df.values)
#kp_df = pd.DataFrame(kp_df_scaled,columns=["KP_"+str(i+1) for i in range(len(emb[0]))])


In [147]:
Data = pd.concat([Data, kp_df], axis=1)


In [148]:
Data.to_pickle("Data130622_F")